In [72]:
import pandas as pd

df = pd.read_csv('../data/whoscored/whoscored_10000_matches.csv')

# 1. DATA CLEANING

In [73]:
print('all matchs:',df.shape)
df = df.loc[df['home_possession'] != 'null']
print('all detailed matchs:',df.shape)

all matchs: (9661, 27)
all detailed matchs: (761, 27)


In [74]:
df['match_id']=df['match_report-href'].str.extract('(\d+)',expand=False).astype(int)
df.drop('match_report-href',axis=1,inplace=True)
df=df.sort_values(by='match_id', ascending=1)

In [75]:
df['home_team_id']=df['home_team-href'].str.extract('(\d+)',expand=False).astype(int)
df['away_team_id']=df['away_team-href'].str.extract('(\d+)',expand=False).astype(int)
df.drop(['home_team-href','away_team-href'],axis=1,inplace=True)
df['tournament_id']=df['tournament-href'].str.extract('Tournaments\/(\d+)',expand=False).astype(int)
df.drop(['tournament-href'],axis=1,inplace=True)

## 1.1. Inspect Tournament INFO

In [76]:
df[['match_report','tournament','tournament_id','country']].groupby(['country','tournament','tournament_id']).count()

match_report
country       tournament                                     tournament_id              
Brazil        Brasileirão - 2016                             95                        1
England       Championship - 2016/2017                       7                       271
              Premier League - 2016/2017                     2                         8
Europe        UEFA Super Cup - 2015/2016                     62                        1
France        Ligue 1 - 2016/2017                            22                      187
International European Championship - 2016 - EURO 1/8 Finals 124                       2
Netherlands   Eredivisie - 2016/2017                         13                      153
Russia        Premier League - 2016/2017                     77                      136
Spain         Supercopa de Espana - 2015/2016                61                        2

### only process tournment England Championahip (252)

In [77]:
df = df.loc[df['tournament_id'] == 7]

## 1.2. convert percentage to fraction

In [78]:
import numpy as np
df['away_possession']=df['away_possession'].replace('%','',regex=True).astype('float')/100
df['home_possession']=df['home_possession'].replace('%','',regex=True).astype('float')/100
df['away_aerial_duel_success']=df['away_aerial_duel_success'].replace('%','',regex=True).astype('float')/100
df['home_aerial_duel_success']=df['home_aerial_duel_success'].replace('%','',regex=True).astype('float')/100
df['home_pass_success']=df['home_pass_success'].replace('%','',regex=True).astype('float')/100
df['away_pass_success']=df['away_pass_success'].replace('%','',regex=True).astype('float')/100

## 1.3. extract home and away goal

In [79]:
df_ht=df['half_time'].str.split(':',expand=True)
df_ht.columns=['home_half_time_goal','away_half_time_goal']
df_ft=df['full_time'].str.split(':',expand=True)
df_ft.columns=['home_full_time_goal','away_full_time_goal']

df = pd.concat([df, df_ht], axis=1, join_axes=[df.index])
df = pd.concat([df, df_ft], axis=1, join_axes=[df.index])

## 1.4. change data type to integer

In [80]:
df['home_full_time_goal']=df['home_full_time_goal'].astype(int)
df['away_full_time_goal']=df['away_full_time_goal'].astype(int)
df['home_half_time_goal']=df['home_half_time_goal'].astype(int)
df['away_half_time_goal']=df['away_half_time_goal'].astype(int)
df['home_shots']=df['home_shots'].astype(int)
df['away_shots']=df['away_shots'].astype(int)
df['home_shots_on_target']=df['home_shots_on_target'].astype(int)
df['away_shots_on_target']=df['away_shots_on_target'].astype(int)
df['home_dribbles_won']=df['home_dribbles_won'].astype(int)
df['away_dribbles_won']=df['away_dribbles_won'].astype(int)
df['home_tackles']=df['home_tackles'].astype(int)
df['away_tackles']=df['away_tackles'].astype(int)

# 2. feature engineering
## 2.1. find goal difference

In [81]:
df['half_time_goal_diff']=df['home_half_time_goal']-df['away_half_time_goal']
df['full_time_goal_diff']=df['home_full_time_goal']-df['away_full_time_goal']

# 3. Training

In [82]:
home_input_columns=['home_team','home_shots','home_shots_on_target',
       'home_pass_success', 
       'home_aerial_duel_success', 
       'home_dribbles_won', 'home_tackles',
       'home_possession', 
       'home_team_id', 'home_half_time_goal',
       'home_full_time_goal']

away_input_columns=['away_shots', 
       'away_shots_on_target', 'away_pass_success',
       'away_aerial_duel_success',
       'away_dribbles_won',
       'away_tackles', 'away_possession', 
       'away_team_id', 
       'away_half_time_goal', 'away_full_time_goal']


home_team_info=df[home_input_columns].groupby('home_team_id').mean()
away_team_info=df[away_input_columns].groupby('away_team_id').mean()
home_team_info.head()

,home_shots,home_shots_on_target,home_pass_success,home_aerial_duel_success,home_dribbles_won,home_tackles,home_possession,home_half_time_goal,home_full_time_goal
home_team_id,,,,,,,,,
19,10.916667,3.666667,0.741667,0.455000,8.500000,19.333333,0.485833,0.416667,1.250000
20,14.818182,4.363636,0.773636,0.510909,8.454545,15.090909,0.530909,0.272727,0.727273
23,15.600000,5.200000,0.790000,0.493000,6.600000,15.800000,0.595000,0.900000,2.200000
24,12.750000,4.750000,0.735000,0.567500,9.166667,18.083333,0.480000,0.666667,1.583333
25,16.363636,4.363636,0.793636,0.504545,6.909091,13.363636,0.584545,0.454545,1.454545


## 3.1. prepare training set and test set

In [83]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

train_columns=['home_shots', 'away_shots', 'home_shots_on_target',
       'away_shots_on_target', 'home_pass_success', 'away_pass_success',
       'home_aerial_duel_success', 'away_aerial_duel_success',
       'home_dribbles_won', 'away_dribbles_won', 'home_tackles',
       'away_tackles', 'home_possession', 'away_possession']
target_column='full_time_goal_diff'
team_id_columns=['home_team_id','away_team_id']

#train_df, test_df = train_test_split(df[train_columns + [target_column] + team_id_columns], test_size = 0.2)
train_size=int(df.shape[0]*0.8)

train_df = df[train_columns + [target_column] + team_id_columns][:train_size]
test_df = df[train_columns + [target_column] + team_id_columns][train_size:]
train_x_df = train_df[train_columns]
train_y_df = train_df[target_column]
test_x_df = test_df[train_columns]
test_y_df = test_df[target_column]

## 3.2. linear regression

In [84]:
regr = LinearRegression()
regr.fit(train_x_df, train_y_df)
print('Coefficients: \n', regr.coef_)
print("Mean squared error: %.2f" % np.mean((regr.predict(test_x_df) - test_y_df) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(test_x_df, test_y_df))
print('Mean absolute error: %.2f' % mean_absolute_error(test_y_df,regr.predict(test_x_df)))


Coefficients: 
 [ -7.65659913e-02   3.64636509e-02   3.78205490e-01  -3.25834696e-01
   2.92556882e+00  -2.77254417e+00   8.95849679e+01   8.76752639e+01
   4.78179929e-02  -1.29729771e-02  -5.29442835e-03  -2.33582235e-02
  -2.31622183e+00   2.31622183e+00]
Mean squared error: 2.29
Variance score: 0.24
Mean absolute error: 1.19


## 3.3. Lasso

In [85]:
from sklearn.linear_model import Lasso
clf = Lasso(alpha=0.1)
clf.fit(train_x_df, train_y_df)

print('Coefficients: \n', clf.coef_)
print("Mean squared error: %.2f" % np.mean((clf.predict(test_x_df) - test_y_df) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % clf.score(test_x_df, test_y_df))
print('Mean absolute error: %.2f' % mean_absolute_error(test_y_df,clf.predict(test_x_df)))



Coefficients: 
 [-0.07660583  0.01893548  0.35922228 -0.30254256 -0.          0.          0.
 -0.          0.03849625 -0.00663751  0.0011944  -0.02509936 -0.          0.        ]
Mean squared error: 2.24
Variance score: 0.26
Mean absolute error: 1.23


## 3.4. Elastic Net

In [86]:
from sklearn.linear_model import ElasticNet

enf = ElasticNet(alpha=0.1)
enf.fit(train_x_df, train_y_df)

print('Coefficients: \n', enf.coef_)
print("Mean squared error: %.2f" % np.mean((enf.predict(test_x_df) - test_y_df) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % enf.score(test_x_df, test_y_df))
print('Mean absolute error: %.2f' % mean_absolute_error(test_y_df,enf.predict(test_x_df)))

Coefficients: 
 [-0.08223228  0.0257419   0.37267927 -0.3193197  -0.          0.          0.
 -0.          0.04375136 -0.01105166  0.00246268 -0.0270821  -0.          0.        ]
Mean squared error: 2.24
Variance score: 0.26
Mean absolute error: 1.23


# 4. Prediction
## 4.1. Whoscored Team ID INFO

In [87]:
home_team_id_map=df.groupby(['home_team_id','home_team']).count().reset_index()[['home_team_id','home_team']]
away_team_id_map=df.groupby(['away_team_id','away_team']).count().reset_index()[['away_team_id','away_team']]


def team_id_to_name(team_id):
    away_team_id_name=away_team_id_map.loc[away_team_id_map['away_team_id']==team_id]    
    if away_team_id_name.size > 0:
        return away_team_id_name['away_team'].values[0]
    
    home_team_id_name=home_team_id_map.loc[home_team_id_map['home_team_id']==team_id]
    if home_team_id_name.size > 0:
        return home_team_id_name['home_team'].values[0]
    return None

for team_id in range(0, 1000):
    team_name = team_id_to_name(team_id)
    if team_name is not None:
        print(str(team_id),':',team_id_to_name(team_id))

19 : Leeds
20 : Derby
23 : Newcastle
24 : Aston Villa
25 : Sheff Wed
94 : Reading
142 : Barnsley
157 : Birmingham
158 : Blackburn
161 : Wolves
165 : Ipswich
166 : Huddersfield
168 : Norwich
170 : Fulham
171 : QPR
174 : Nottingham Forest
181 : Preston
182 : Bristol City
188 : Cardiff
189 : Brentford
194 : Wigan
210 : Rotherham
211 : Brighton


In [88]:
home_team_info.head()

,home_shots,home_shots_on_target,home_pass_success,home_aerial_duel_success,home_dribbles_won,home_tackles,home_possession,home_half_time_goal,home_full_time_goal
home_team_id,,,,,,,,,
19,10.916667,3.666667,0.741667,0.455000,8.500000,19.333333,0.485833,0.416667,1.250000
20,14.818182,4.363636,0.773636,0.510909,8.454545,15.090909,0.530909,0.272727,0.727273
23,15.600000,5.200000,0.790000,0.493000,6.600000,15.800000,0.595000,0.900000,2.200000
24,12.750000,4.750000,0.735000,0.567500,9.166667,18.083333,0.480000,0.666667,1.583333
25,16.363636,4.363636,0.793636,0.504545,6.909091,13.363636,0.584545,0.454545,1.454545


## 4.2. Test Prediction

In [89]:
#try predict home team 19 vs away team 20
home_team_info_1=home_team_info.reset_index()
home_team_features=home_team_info_1.loc[home_team_info_1['home_team_id']==19]
del home_team_info_1
home_team_features

,home_team_id,home_shots,home_shots_on_target,home_pass_success,home_aerial_duel_success,home_dribbles_won,home_tackles,home_possession,home_half_time_goal,home_full_time_goal
0,19,10.916667,3.666667,0.741667,0.455,8.5,19.333333,0.485833,0.416667,1.25


In [90]:
away_team_info_1=away_team_info.reset_index()
away_team_features=away_team_info_1.loc[away_team_info_1['away_team_id']==20]
del away_team_info_1
away_team_features

,away_team_id,away_shots,away_shots_on_target,away_pass_success,away_aerial_duel_success,away_dribbles_won,away_tackles,away_possession,away_half_time_goal,away_full_time_goal
1,20,11.0,3.818182,0.744545,0.502727,7.454545,15.363636,0.480909,0.454545,1.090909


In [91]:
features = pd.concat([home_team_features.reset_index(), away_team_features.reset_index()], axis=1)[train_columns]
features

,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_pass_success,away_pass_success,home_aerial_duel_success,away_aerial_duel_success,home_dribbles_won,away_dribbles_won,home_tackles,away_tackles,home_possession,away_possession
0,10.916667,11.0,3.666667,3.818182,0.741667,0.744545,0.455,0.502727,8.5,7.454545,19.333333,15.363636,0.485833,0.480909


In [92]:
#normalize features
features['total_possession']=features['home_possession']+features['away_possession']
features['home_possession']=features['home_possession']/features['total_possession']
features['away_possession']=features['away_possession']/features['total_possession']
features.drop('total_possession',axis=1,inplace=True)
features

,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_pass_success,away_pass_success,home_aerial_duel_success,away_aerial_duel_success,home_dribbles_won,away_dribbles_won,home_tackles,away_tackles,home_possession,away_possession
0,10.916667,11.0,3.666667,3.818182,0.741667,0.744545,0.455,0.502727,8.5,7.454545,19.333333,15.363636,0.502547,0.497453


In [93]:
#test_x_df.head()
regr.predict(features)

array([-3.5697853])

## 4.3. Make prediction with all test data set

In [94]:
def predict_diff(home_team_id, away_team_id):
    home_team_info_1=home_team_info.reset_index()
    home_team_features=home_team_info_1.loc[home_team_info_1['home_team_id']==home_team_id]
    del home_team_info_1
    away_team_info_1=away_team_info.reset_index()
    away_team_features=away_team_info_1.loc[away_team_info_1['away_team_id']==away_team_id]
    del away_team_info_1
    away_team_features
    features = pd.concat([home_team_features.reset_index(), away_team_features.reset_index()], axis=1)[train_columns]
    features['total_possession']=features['home_possession']+features['away_possession']
    features['home_possession']=features['home_possession']/features['total_possession']
    features['away_possession']=features['away_possession']/features['total_possession']
    features.drop('total_possession',axis=1,inplace=True)
    return regr.predict(features)

In [105]:
test_df_predict = test_df[['home_team_id','away_team_id','full_time_goal_diff']]
test_df_predict['predict_goal_diff']=0
test_df_predict.head()

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,home_team_id,away_team_id,full_time_goal_diff,predict_goal_diff
3645,157,211,-1,0
178,210,94,-1,0
7550,210,1786,-1,0
2179,157,142,-3,0
5340,194,94,-3,0


In [96]:
for index, row in test_df_predict.iterrows():
    predicted_diff=predict_diff(row['home_team_id'],row['away_team_id'])[0]    
    #print(row['full_time_goal_diff'],":",predicted_diff)
    row['predict_goal_diff']=predicted_diff

In [97]:
test_df_predict

,home_team_id,away_team_id,full_time_goal_diff,predict_goal_diff
3645,157,211,-1,4
178,210,94,-1,-6
7550,210,1786,-1,-4
2179,157,142,-3,0
5340,194,94,-3,0
3315,158,94,-1,0
1248,25,171,1,0
8333,161,171,-1,-2
2978,158,166,0,-7
1518,161,20,-1,-1


In [98]:
import clean_sofifa_data as sofifa
sofifa_df = sofifa.get_cleaned_sofifa_data()

In [99]:
df['datetime']=pd.to_datetime(df['date'])
df[['home_team_id','home_team','datetime']].head()

,home_team_id,home_team,datetime
7535,170,Fulham,2016-08-05
1799,157,Birmingham,2016-08-06
788,158,Blackburn,2016-08-06
5650,182,Bristol City,2016-08-06
5907,20,Derby,2016-08-06


In [108]:
import datetime
sofifa_df.whoscored_team_id
sofifa_df.loc[(sofifa_df.whoscored_team_id == 170) & (sofifa_df.last_update_datetime<'2016-08-05')].sort_values(by='last_update_datetime', ascending=0).head(1)

,clubs,overall,attack,midfield,defence,starting_avg_age,whole_team_avg_age,build_up_play_speed,build_up_play_dribbling,build_up_play_passing,...,chance_creation_positioning,defence_pressure,defence_aggression,defence_team_width,defence_defender_line,sofifa_team_id,transfer_budget_total,fifa_edition,last_update_datetime,whoscored_team_id
2201,Fulham,70,71,70,69,24.3,23.8,39,41,57,...,Organised,31,37,39,Cover,144,1000000.0,FIFA 16,2016-08-04,170
